# Imports

In [41]:
from aws_utils import s3_client, bucket_name, fs
import glob, os, io, pickle
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import TensorDataset, DataLoader, random_split, Dataset, WeightedRandomSampler

# Load in Data

In [42]:
key = "final_data.pkl"                           # object lives at s3://maniks-chb-mit/final_data.pkl

buf = io.BytesIO()                              # in‑memory buffer
s3_client.download_fileobj(bucket_name, key, buf)
buf.seek(0)                                     # rewind

data = pickle.load(buf)                         # un‑pickle
X, y = data["X"], data["y"]

print("Loaded:", X.shape, y.shape)               # e.g.  torch.Size([1601, 22, 1471])  (1601,)

Loaded: torch.Size([14464, 22, 1471]) torch.Size([14464])


In [47]:
import torch

# -------------------------------------------------------------
# X : Tensor of shape (N, …)          – your spectrograms / epochs
# y : Tensor of shape (N,)            – 0 = inter‑ictal, 1 = pre‑ictal
# -------------------------------------------------------------
torch.manual_seed(42)                 # ← for reproducibility; optional

# 1) find indices of each class
idx_int = (y == 0).nonzero(as_tuple=True)[0]   # inter‑ictal indices
idx_pre = (y == 1).nonzero(as_tuple=True)[0]   # pre‑ictal indices

# 2) decide how many inter‑ictal samples to keep
#    here: keep as many inter‑ictal as pre‑ictal ⇒ perfectly balanced
k = len(idx_pre)

perm      = torch.randperm(len(idx_int))       # shuffle
idx_int_k = idx_int[perm[:k]]                  # first k indices

# 3) combine & shuffle (optional but recommended)
idx_keep = torch.cat([idx_pre, idx_int_k])
idx_keep = idx_keep[torch.randperm(len(idx_keep))]

# 4) slice tensors
X_bal = X[idx_keep]
y_bal = y[idx_keep]

print("after balancing:", torch.bincount(y_bal))
# tensor([k, k])           ← equal counts now


after balancing: tensor([3231, 3231])


In [52]:
dataset = TensorDataset(X_bal, y_bal)
class_sample_count = torch.bincount(y_bal)


In [53]:
print(class_sample_count)

tensor([3231, 3231])


In [54]:
train_size = int(0.8 * len(dataset))
test_size = len(dataset) - train_size
train_data, test_data = random_split(dataset, [train_size, test_size])

# Model Creation

In [55]:
class CNNUnit(nn.Module):
    def __init__(self, in_channels, out_channels=16):
        super(CNNUnit, self).__init__()
        
        # First 3x3 Conv Layer
        self.conv1 = nn.Conv2d(in_channels, out_channels, kernel_size=3, padding=1)
        self.bn1 = nn.BatchNorm2d(out_channels)
        self.gelu1 = nn.GELU()
        
        # Second 3x3 Conv Layer 
        self.conv2 = nn.Conv2d(out_channels, out_channels, kernel_size=3, padding=1)
        self.bn2 = nn.BatchNorm2d(out_channels)
        self.gelu2 = nn.GELU()

        # Third 3x3 Conv Layer with Stride 2 (Downsampling)
        self.conv3 = nn.Conv2d(out_channels, out_channels, kernel_size=3, stride=2, padding=1)
        self.bn3 = nn.BatchNorm2d(out_channels)
        self.gelu3 = nn.GELU()

    def forward(self, x):

        # First Conv2D + GELU + BatchNorm
        x = self.gelu1(self.bn1(self.conv1(x)))  
        
        # Second Conv2D + GELU + BatchNorm
        x = self.gelu2(self.bn2(self.conv2(x)))  
        
        # Third Conv2D (Stride=2 for downsampling) + GELU + BatchNorm
        x = self.gelu3(self.bn3(self.conv3(x)))  
        return x

In [56]:
class CNNLayer(nn.Module):
    def __init__(self, in_channels, out_channels=24):
        super(CNNLayer, self).__init__()

        self.conv1 = nn.Conv2d(in_channels, out_channels, kernel_size=3, stride=2, padding=1)
        self.bn1 = nn.BatchNorm2d(out_channels)
        self.gelu1 = nn.GELU()
        
    def forward(self, x):

        x = self.gelu1(self.bn1(self.conv1(x)))
        return x

In [57]:
class LocalInformationLayer(nn.Module):
    def __init__(self, in_channels=24):
        super(LocalInformationLayer, self).__init__()
        self.conv = nn.Conv2d(in_channels, in_channels, kernel_size=3, padding=1, bias=False)

    def forward(self, x):
        return self.conv(x) + x

In [58]:
class SMHSA(nn.Module):
    def __init__(self, dim, heads=8, kernel_size=3, stride=2, bias=True):
        super(SMHSA, self).__init__()

        assert dim % heads == 0, "dim must be divisible by heads"

        self.dim = dim
        self.heads = heads
        self.scale = (dim // heads) ** -0.5  # Scaling factor for attention

        # Linear layers to compute Query, Key, and Value
        self.to_qkv = nn.Linear(dim, dim * 3, bias=False)

        # Depth-wise convolution for Q and K downsampling
        self.spatial_reduction = nn.Conv2d(
            dim, dim, kernel_size=kernel_size, stride=stride, padding=kernel_size//2, 
            groups=dim, bias=False
        )

        # Initialize bias with proper shape for attention
        self.bias = nn.Parameter(torch.zeros(1, heads, 1, 1)) if bias else None

        # Final projection after self-attention
        self.to_out = nn.Linear(dim, dim)

    def forward(self, x):
        B, C, H, W = x.shape
        
        # Reshape input to sequence format
        x = x.permute(0, 2, 3, 1).reshape(B, H * W, C)  # (B, seq_len, C)

        # Compute Query, Key, and Value
        qkv = self.to_qkv(x)
        q, k, v = qkv.chunk(3, dim=-1)

        # Reshape for multi-head attention
        q = q.view(B, -1, self.heads, C // self.heads).permute(0, 2, 1, 3)
        k = k.view(B, -1, self.heads, C // self.heads).permute(0, 2, 1, 3)
        v = v.view(B, -1, self.heads, C // self.heads).permute(0, 2, 1, 3)

        # 🔹 **Fix: Ensure correct reshaping before Conv2D**
        q = q.permute(0, 1, 3, 2).reshape(B, C, H, W)
        k = k.permute(0, 1, 3, 2).reshape(B, C, H, W)

        # Apply spatial reduction
        q = self.spatial_reduction(q)
        k = self.spatial_reduction(k)

        # Get reduced spatial dimensions after Conv2D
        H_reduced, W_reduced = q.shape[2], q.shape[3]

        # Reshape back for attention
        q = q.reshape(B, self.heads, C // self.heads, H_reduced * W_reduced).permute(0, 1, 3, 2)
        k = k.reshape(B, self.heads, C // self.heads, H_reduced * W_reduced).permute(0, 1, 3, 2)

        # Compute attention scores
        attn = torch.matmul(q, k.transpose(-2, -1)) * self.scale

        # Apply bias if enabled
        if self.bias is not None:
            # Expand bias to match attention dimensions
            bias_expanded = self.bias.expand(B, self.heads, H_reduced * W_reduced, H_reduced * W_reduced)
            attn = attn + bias_expanded

        attn = F.softmax(attn, dim=-1)

        # Apply attention to Value
        out = torch.matmul(attn, v)

        # Reshape output back to spatial format
        out = out.permute(0, 2, 1, 3).reshape(B, H_reduced * W_reduced, C)
        out = self.to_out(out)
        out = out.reshape(B, H_reduced, W_reduced, C).permute(0, 3, 1, 2)

        return out


In [59]:
class RFFN(nn.Module):
    def __init__(self, dim, expansion=4):
        super(RFFN, self).__init__()
        
        hidden_dim = dim * expansion

        self.conv1 = nn.Conv2d(dim, hidden_dim, kernel_size=1, padding=0, bias=False)
        self.bn1 = nn.BatchNorm2d(hidden_dim)
        self.gelu1 = nn.GELU()

        self.conv2 = nn.Conv2d(hidden_dim, hidden_dim, kernel_size=3, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(hidden_dim)
        self.gelu2 = nn.GELU()

        self.conv3 = nn.Conv2d(hidden_dim, dim, kernel_size=1, padding=0, bias=False)
        self.bn3 = nn.BatchNorm2d(dim)

    def forward(self, x):

        residual = x
        x = self.gelu1(self.bn1(self.conv1(x)))
        x = self.gelu2(self.bn2(self.conv2(x)))
        x = self.bn3(self.conv3(x))

        return x + residual
        

In [60]:
import torch
import torch.nn as nn

class FC(nn.Module):
    def __init__(self, in_features):
        """
        Fully Connected (FC) Layer for Classification

        Args:
        - in_features: Number of input features (from GAP output, i.e., number of channels)
        """
        super(FC, self).__init__()

        self.pool = nn.AdaptiveAvgPool2d((1, 1))  # Global Average Pooling

        self.fc1 = nn.Linear(in_features, 512)  # First Fully Connected Layer
        self.relu1 = nn.ReLU()
        self.dropout = nn.Dropout(0.5)  # Dropout to prevent overfitting
        self.fc2 = nn.Linear(512, 2)  # Output Layer (Binary Classification)

    def forward(self, x):
        x = self.pool(x)  # Apply Global Average Pooling
        x = torch.flatten(x, start_dim=1)  # Flatten from (B, C, 1, 1) -> (B, C)
        x = self.relu1(self.fc1(x))  # First Fully Connected Layer + ReLU
        x = self.dropout(x)  # Apply Dropout
        x = self.fc2(x)  # Final Output Layer
        return x


In [61]:
class TGCNN(nn.Module):
    def __init__(self):
        super(TGCNN, self).__init__()

        # CNN Layers
        self.cnn_unit = CNNUnit(in_channels=22, out_channels=16)
        self.cnn_layer1 = CNNLayer(in_channels=16, out_channels=24)
        self.cnn_layer2 = CNNLayer(in_channels=24, out_channels=32)

        # Transformer Block 1
        self.LIL1 = LocalInformationLayer(in_channels=24)
        self.norm1a = nn.LayerNorm([24, 2, 320])

        self.multiheaded_att1 = SMHSA(dim=24, stride=1)  # Adjust stride if needed
        self.norm1b = nn.LayerNorm([24, 2, 320])
        self.recurrent1 = RFFN(dim=24)

        # Transformer Block 2
        self.LIL2 = LocalInformationLayer(in_channels=32)
        self.norm1b = nn.LayerNorm([32, 1, 160])
        self.multiheaded_att2 = SMHSA(dim=32, stride=1)  # Adjust stride if needed
        self.norm1b = nn.LayerNorm([32, 1, 160])
        self.recurrent2 = RFFN(dim=32)

        # Global Average Pooling + Fully Connected Layer
        self.fc = FC(in_features=32)  # Matches last CNN layer's output channels

    def forward(self, x):
        x = x.unsqueeze(2)
        x = self.cnn_unit(x)      
        x = self.cnn_layer1(x)    
        x = self.LIL1(x)          
        x = self.multiheaded_att1(x)
        x = self.recurrent1(x)    
        x = self.cnn_layer2(x)    
        x = self.LIL2(x)          
        x = self.multiheaded_att2(x)
        x = self.recurrent2(x)    
        x = self.fc(x)

        return x


In [62]:
data = torch.rand(100,22,1280)
model = TGCNN()
output = model(data)
print(output.shape)

torch.Size([100, 2])


# Load in Data

# ML

In [63]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
import numpy as np
from sklearn.model_selection import train_test_split

# Check for GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# # Example Data (Ensure your real data is loaded correctly)
# spectrogram_data = np.random.rand(1000, 28, 5, 1280)  # Simulated input
# y = np.random.randint(0, 2, (1000,))  # Simulated binary classification labels

# # Train-Test Split
# X_train, X_test, y_train, y_test = train_test_split(spectrogram_data, y, test_size=0.2, random_state=42)

# # Convert to PyTorch Tensors
# X_train = torch.tensor(X_train, dtype=torch.float32).to(device)
# X_test = torch.tensor(X_test, dtype=torch.float32).to(device)
# y_train = torch.tensor(y_train, dtype=torch.long).to(device)
# y_test = torch.tensor(y_test, dtype=torch.long).to(device)

# # Create DataLoader objects
batch_size = 32
# train_dataset = TensorDataset(X_train, y_train)
# test_dataset = TensorDataset(X_test, y_test)

train_loader = DataLoader(train_data, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_data, batch_size=batch_size, shuffle=False)

# Model, Loss Function, and Optimizer
model = TGCNN().to(device)  # Move model to GPU if available
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Training Parameters
num_epochs = 20  # Adjust based on dataset size
print_interval = 5  # Print every 5 batches

# 🚀 Training Loop
for epoch in range(num_epochs):
    model.train()  # Set model to training mode
    total_loss = 0
    correct = 0
    total = 0
    
    for batch_idx, (inputs, targets) in enumerate(train_loader):
        inputs, targets = inputs.to(device), targets.to(device)
        
        optimizer.zero_grad()  # Reset gradients
        outputs = model(inputs)  # Forward pass
        
        loss = criterion(outputs, targets)  # Compute loss
        loss.backward()  # Backpropagation
        
        nn.utils.clip_grad_norm_(model.parameters(), max_norm=5)  # Optional: Gradient clipping
        optimizer.step()  # Update weights
        
        total_loss += loss.item()
        predicted = torch.argmax(outputs, dim=1)
        correct += (predicted == targets).sum().item()
        total += targets.size(0)

        if batch_idx % print_interval == 0:
            print(f"Epoch [{epoch+1}/{num_epochs}], Step [{batch_idx+1}/{len(train_loader)}], Loss: {loss.item():.4f}")

    # Print epoch summary
    print(f"🔥 Epoch [{epoch+1}/{num_epochs}] -> Loss: {total_loss/len(train_loader):.4f}, Accuracy: {100 * correct / total:.2f}%")

# Save the trained model
torch.save(model.state_dict(), "TGCNN_model.pth")
print("✅ Model saved successfully!")

# 🚀 Evaluation Function
def evaluate_model(model, test_loader):
    model.eval()  # Set model to evaluation mode
    correct = 0
    total = 0
    all_preds = []
    all_labels = []
    
    with torch.no_grad():  # Disable gradient calculation
        for inputs, targets in test_loader:
            inputs, targets = inputs.to(device), targets.to(device)
            outputs = model(inputs)
            
            predicted = torch.argmax(outputs, dim=1)
            correct += (predicted == targets).sum().item()
            total += targets.size(0)
            
            all_preds.extend(predicted.cpu().numpy())
            all_labels.extend(targets.cpu().numpy())

    accuracy = 100 * correct / total
    print(f"🎯 Test Accuracy: {accuracy:.2f}%")
    return accuracy

# Run Evaluation
evaluate_model(model, test_loader)


Using device: cuda
Epoch [1/20], Step [1/162], Loss: 0.7061
Epoch [1/20], Step [6/162], Loss: 0.6934
Epoch [1/20], Step [11/162], Loss: 0.6549
Epoch [1/20], Step [16/162], Loss: 0.7899
Epoch [1/20], Step [21/162], Loss: 0.6281
Epoch [1/20], Step [26/162], Loss: 0.5240
Epoch [1/20], Step [31/162], Loss: 0.4939
Epoch [1/20], Step [36/162], Loss: 0.5736
Epoch [1/20], Step [41/162], Loss: 0.7252
Epoch [1/20], Step [46/162], Loss: 0.7063
Epoch [1/20], Step [51/162], Loss: 0.4075
Epoch [1/20], Step [56/162], Loss: 0.5038
Epoch [1/20], Step [61/162], Loss: 0.6376
Epoch [1/20], Step [66/162], Loss: 0.4342
Epoch [1/20], Step [71/162], Loss: 0.6965
Epoch [1/20], Step [76/162], Loss: 0.4697
Epoch [1/20], Step [81/162], Loss: 0.6698
Epoch [1/20], Step [86/162], Loss: 0.4742
Epoch [1/20], Step [91/162], Loss: 0.8918
Epoch [1/20], Step [96/162], Loss: 0.5903
Epoch [1/20], Step [101/162], Loss: 0.4362
Epoch [1/20], Step [106/162], Loss: 0.4012
Epoch [1/20], Step [111/162], Loss: 0.4998
Epoch [1/20], 

47.33178654292343